In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import re
import time

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.mars
collection = db.items

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Scrape NASA News

In [5]:
# URL
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [6]:
#Path to get titles of articles
title = '//div[@class="content_title"]/a'

#Reading the results  
nasa_title = browser.find_by_xpath(title).first.text


In [7]:
#Scrape teaser

teaser = '//div[@class="article_teaser_body"]'
nasa_teaser = browser.find_by_xpath(teaser).first.text


## JPL Mars Space Images - Featured Image

In [8]:
#JPL full-image
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
browser.find_by_id('full_image').click()
featured_image_url = browser.find_by_css('.fancybox-image').first['src']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA20263_ip.jpg'

## Twitter Weather

In [ ]:
#Mars twitter
twitter = "https://twitter.com/marswxreport?lang=en"
response = requests.get(twitter)

# Create BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# find a weather tweet
mars_weather=soup.find(string=re.compile(", high"))
print(mars_weather)

## Mars Facts

In [ ]:
# We visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet
import pandas as pd
marsfacts = 'https://space-facts.com/mars/'
tables = pd.read_html(marsfacts)
tables

In [ ]:
# Now let's slice off the dataframe using normal indexing.
df = tables[0]
df.columns = ['Col1', 'Col2']
df.set_index('Col1', inplace=True)
df

In [ ]:
#Converting the data to a HTML table string and stripping unwanted newlines to clean up the table

mars_html = df.to_html()
mars_html.replace("\n", " ")

## Mars Hemispheres

In [11]:
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemisphere_image_urls = []
browser.visit(usgs_url)
for i in range(4):
    dictionary = {}
    hemis = browser.find_by_tag('h3')[i].text
    browser.find_by_css('.thumb')[i].click()
    img = browser.find_by_text('Sample')['href']
    browser.back()
    dictionary['hemisphere'] = hemis
    dictionary['img'] = img
    hemisphere_image_urls.append(dictionary)
print(hemisphere_image_urls)

[{'hemisphere': 'Cerberus Hemisphere Enhanced', 'img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'hemisphere': 'Schiaparelli Hemisphere Enhanced', 'img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'hemisphere': 'Syrtis Major Hemisphere Enhanced', 'img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'hemisphere': 'Valles Marineris Hemisphere Enhanced', 'img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [ ]:
import shutil
import requests
for img in images_url:
    response = requests.get(img, stream=True) 
    with open('img.jpg', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)

In [ ]:
# Displaying the first image with IPython.display
from IPython.display import Image
Image(url='img.jpg')